# Performance Benchmarking: Pandas DataFrame vs List of Dictionaries

Benchmark performance of pandas.DataFrame against a native python list for element-wise assignment.

## Problem

While in the initial stages of a project, sometimes we have to choose between storing data with pandas dataframes or in native python lists of dictionaries. Both data structures look similar enough to perform the same tasks - we can even look at lists of dictionaries as simply a less complex pandas dataframe (each row in a dataframe corresponds to each dictionary in the list). 

The question then arises: given the increased complexity and overhead of a pandas dataframe, is it true then that we should always default to using python lists of dictionaries when performance is the primary consideration? 

The answer, it would seem, is no. This we demonstrate by examining the use case of element-wise assignment**.

## Setup

First, some scaffolding. For the dataset, we use the new_york_hotels.csv dataset.

In [17]:
import pandas as pd
import numpy as np
import csv
import math
from timeit import Timer

df = pd.read_csv('new_york_hotels.csv', encoding='cp1252')  # dataframe
l  = df.to_dict('records')                                  # list of dictionaries

For this comparison, we will be using the haversine function used by [Sofia Heisler in Pycon2017](https://github.com/s-heisler/pycon2017-optimizing-pandas/blob/master/pyCon%20materials/PyCon%20un-sad%20Pandas.ipynb). This function is a vectorised implementation, optimised for Pandas DataFrames. 

In [10]:
def df_haversine(lat1, lon1, lat2, lon2):
  miles_constant = 3959
  lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
  dlat = lat2 - lat1
  dlon = lon2 - lon1
  a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
  c = 2 * np.arcsin(np.sqrt(a))
  mi = miles_constant * c
  return mi

An alternative implementation of the haversine function has been optimised for use with lists.

In [11]:
def for_haversine(lat1, lon1, lat2, lon2):
  miles_constant = 3959
  lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
  dlat = lat2 - lat1
  dlon = lon2 - lon1
  a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
  c = 2 * math.asin(math.sqrt(a))
  mi = miles_constant * c
  return mi

## Comparison

### Pandas DataFrame

In [29]:
def timing_loop():
    df['distance'] = df_haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

df_time = min(Timer(timing_loop).repeat(10, 100))
print(f'{df_time:.3f}s for best run.')

0.033s for best run.


### Python List

In [30]:
def timing_loop():
    for o in l:
        lat = o['latitude']
        lon = o['longitude']
        o['distance'] = for_haversine(40.671, -73.985, lat, lon)

list_time = min(Timer(timing_loop).repeat(10, 100))
print(f'{list_time:.3f}s for best run.')

0.343s for best run.


## Results

From the above, we can see that the dataframe implementation is about 10X faster than the list implementation. Which is surprising. Some further digging establishes the reasons for this - pandas implements additional optimisations in many use cases, some of these in C code. Such optimisations like [vectorization](https://datascience.blog.wzb.eu/2018/02/02/vectorization-and-parallelization-in-python-with-numpy-and-pandas/) add a level of power to pandas dataframes that would be hard and/or time-consuming to emulate while using a list of dictionaries. 

## References

- https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6
- https://github.com/s-heisler/pycon2017-optimizing-pandas
- https://leadsift.com/loop-map-list-comprehension/
- http://effbot.org/zone/python-list.htm

** Element-wise assignment in this case refers to the iterating of a list of dictionaries, running computations on the values of each individual dictionary, and then assigning the result of that computation onto the same dictionary. 